In [28]:
! pip install tensorflow scikit-learn pandas -q

In [29]:
#importaciòn de librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [30]:
#lectura de dataset
data = pd.read_csv('teleCust1000t.csv')
data.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside,custcat
0,2,13,44,1,9,64.0,4,5,0.0,0,2,1
1,3,11,33,1,7,136.0,5,5,0.0,0,6,4
2,3,68,52,1,24,116.0,1,29,0.0,1,2,3
3,2,33,33,0,12,33.0,2,0,0.0,1,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4,3


In [31]:
# Separar características y etiquetas
X = data.drop(columns=['custcat'])
y = data['custcat']

In [32]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [39]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [40]:
# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
# Construir el modelo
model = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(len(pd.Series(y).unique()), activation='softmax')  # Salida con tantas neuronas como categorías
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2461 - loss: 1.4203 - val_accuracy: 0.2929 - val_loss: 1.3535
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2921 - loss: 1.3563 - val_accuracy: 0.3429 - val_loss: 1.3361
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3924 - loss: 1.3258 - val_accuracy: 0.3714 - val_loss: 1.3198
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4064 - loss: 1.3003 - val_accuracy: 0.3643 - val_loss: 1.3022
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4163 - loss: 1.2957 - val_accuracy: 0.3786 - val_loss: 1.2830
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3884 - loss: 1.2953 - val_accuracy: 0.3357 - val_loss: 1.2721
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3986 - loss: 1.2666 - val_accuracy: 0.3714 - val_loss: 1.2548
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4326 - loss: 1.2554 - val_accuracy: 0.3857 - val_loss

In [42]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)

# Predicciones
y_pred = model.predict(X_test_scaled)
y_pred_classes = y_pred.argmax(axis=1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3698 - loss: 1.3791 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [43]:
# Reporte de clasificación
classification_rep = classification_report(y_test, y_pred_classes)

# Imprimir resultados
print(f"Pérdida en el conjunto de prueba: {loss}")
print(f"Precisión en el conjunto de prueba: {accuracy}")
print("Reporte de clasificación:")
print(classification_rep)

Pérdida en el conjunto de prueba: 1.3473846912384033
Precisión en el conjunto de prueba: 0.39666667580604553
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.43      0.50      0.47        80
           1       0.32      0.28      0.30        65
           2       0.42      0.38      0.40        84
           3       0.39      0.41      0.40        71

    accuracy                           0.40       300
   macro avg       0.39      0.39      0.39       300
weighted avg       0.39      0.40      0.39       300

